In [ ]:
import json
from dataclasses import dataclass
from typing import Optional

@dataclass
class MelanomaPatientData:
    question2: int  # Age
    question3: str # Gender
    question56: Optional[str] = None  # Personal history of skin cancer?
    question59: Optional[str] = None  # Family history of melanoma?
    question60: Optional[str] = None  # Number of relatives with skin cancer


def get_melanoma_screening_recommendation(patient_data: MelanomaPatientData) -> str:
    """
    Analyzes patient data to determine melanoma screening recommendations based on provided guidelines.

    Args:
        patient_data: An instance of MelanomaPatientData containing patient responses.

    Returns:
        A string describing the melanoma screening recommendation.
    """

    # Check for missing required data
    if patient_data.question2 is None or patient_data.question3 is None:
        return "ERROR: Missing required data: Age and Gender are mandatory."

    age = patient_data.question2
    gender = patient_data.question3
    personal_skin_cancer_history = patient_data.question56
    family_melanoma_history = patient_data.question59
    num_relatives_with_skin_cancer = patient_data.question60


    # High-risk patients screening (Annual full-body skin exam by expert)
    if personal_skin_cancer_history == "Item 2": # Yes to personal history of skin cancer
        return "Indicated for melanoma screening: Annual full-body skin examination by a clinician with skin expertise due to personal history of skin cancer."

    if family_melanoma_history == "Item 2": # Yes to family history of melanoma
        # Considering "family history yes" as a risk factor for annual screening, simplifying guideline interpretation for form questions.
        # More nuanced interpretation of family history guideline may require more detailed questions.
        return "Indicated for melanoma screening: Annual full-body skin examination by a clinician with skin expertise due to family history of melanoma."


    # Other patients - Opportunistic screening
    if gender == "Item 2" and age >= 50: # White male 50 and above (assuming general population for now, ideally ethnicity should be considered)
        return "Indicated for opportunistic melanoma screening: Focused skin examination, especially on back, chest, abdomen, head, neck, and arms during routine visits, due to increased melanoma mortality risk in White males over 50.  Please note that for more precise risk assessment, information on race/ethnicity is recommended."


    # General advice for all other patients (average risk based on available data)
    return "Not indicated for routine melanoma screening based on high-risk criteria from the provided data. Recommended: Routine skin self-examination and awareness of melanoma risk factors and signs. Opportunistic skin examination during routine visits is suggested. Please consult your physician if you have any concerning skin lesions."


def read_json_from_file(file_path: str) -> MelanomaPatientData:
    """
    Reads patient data from a JSON file and returns a MelanomaPatientData object.
    """
    with open(file_path, 'r', encoding='utf-8') as f: # added encoding to handle potential persian characters
        data = json.load(f)
    return MelanomaPatientData(
        question2=int(data.get("question2")) if data.get("question2") else None,
        question3=data.get("question3"),
        question56=data.get("question56"),
        question59=data.get("question59"),
        question60=data.get("question60"),
    )


if __name__ == "__main__":
    file_path = "patient_melanoma_data.json"  # Replace with your json file path

    # Example patient data JSON file (patient_melanoma_data.json) - create this file for testing
    # {
    #     "question1": ["Item 13"], # Melanoma screening selected
    #     "question2": "60",
    #     "question3": "Item 2", # Male
    #     "question56": "Item 1", # No personal history of skin cancer
    #     "question59": "Item 2", # Yes to family history
    #     "question60": "Item 1" # One relative
    # }

    try:
        patient_data = read_json_from_file(file_path)
        screening_recommendation = get_melanoma_screening_recommendation(patient_data)
        print(screening_recommendation)

    except FileNotFoundError:
        print(f"Error: File not found at path: {file_path}")
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in file: {file_path}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")